# Notebook für DeDUG-Treffen, 20.04.2018

In [1]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
              'start_slideshow_at': 'selected',
              'slidenumber' : 'true',
        'after_body' : 'ibm_closing.html'
        
})

{u'after_body': 'ibm_closing.html',
 u'slidenumber': 'true',
 u'start_slideshow_at': 'selected',
 u'transition': u'none'}

In [1]:
# Datei hat Variable "LocalDB2password" mit zugewiesenem Passwort
filename = "myvariable.py"

In [4]:
%run $filename

# SQL-Rekursion in Db2
***

## Dr. Henrik Loeser, 20.04.2018
Offering Manager & Developer Advocate<br>
Focus: IBM Cloud with Data & Analytics, Data Security, Privacy & Compliance


* Email: hloeser@de.ibm.com
* Twitter: @data_henrik
* Blog: http://blog.4loeser.net
* LinkedIn: http://de.linkedin.com/in/henrikloeser
* GitHub: https://github.com/data-henrik




# Agenda
***
* Rekursion
* SQL Rekursion
* Common Table Expression (CTE)




# Rekursion
***
* Lateinisch: recurrere ("zurücklaufen")
* Wikipedia:
  > Als Rekursion bezeichnet man den abstrakten Vorgang, dass Regeln auf ein Produkt, das sie hervorgebracht haben, von neuem angewandt werden. Hierdurch entstehen potenziell unendliche Schleifen.
* Kunst: "Mise en abyme", ohne Boden, unendlich
* Beispiele: Sauerteig, Droste-Effekt, mehrere Spiegel
* rekursive Abkürzungen: "IBM bietet mehr", GNU, 




![](https://upload.wikimedia.org/wikipedia/commons/7/77/La_vache_qui_rit.jpg)

![](http://web.cs.ucla.edu/~klinger/dorene/Gif/escher-hands.gif)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Droste_Cacao_Alcalinise_blikje%2C_foto4.JPG/800px-Droste_Cacao_Alcalinise_blikje%2C_foto4.JPG)

# SQL-Rekursion
***
* Berechnung von Materiallisten (bill of material)
* Ersatz für VIEW-Definitionen




# SQL Rekursion und DB2
* Common Table Expression
* CONNECT BY (Oracle-Syntax)

# SQL Rekursion und DB2: CTE

* Generelle Syntax:   
`    WITH with_query [, ...]`   
`     SELECT...`
* with_query:   
`query_name [ (column_name [,...]) ] AS (SELECT ...)`
* Eine oder mehrere Tabellendefinitionen ("query_name") sind möglich
* Wechselseitige Abhängigkeiten nicht erlaubt


# Zusammenfassung
***
* siehe Einführung zu Rekursion
* falls Sie hier lesen, war irgendwo eine Abbruchbedingung erfolgreich
* Rekursion in SQL über Common Table Expression oder Anbieter-spezifische Konstrukte
* Einfach, ein System in die Knie zu zwingen
  

In [5]:
# loads the SQL magic extensions
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
# Connect to Db2
%sql db2+ibm_db://hloeser:$LocalDB2password@localhost:50000/SAMPLE        

u'Connected: hloeser@SAMPLE'

### Berechnung der Fakultät
als einfaches Beispiel

In [7]:
%%sql
WITH  temp (n, fact) AS 
(values( 0, 1 )
  UNION ALL 
 SELECT n+1, (n+1)*fact FROM temp 
        WHERE n < 9)
SELECT * FROM temp


 * db2+ibm_db://hloeser:***@localhost:50000/SAMPLE
Done.


n,fact
0,1
1,1
2,2
3,6
4,24
5,120
6,720
7,5040
8,40320
9,362880


## Keine Faktualität, aber Summen

In [8]:
%%sql 
with mytable(level,mysum) as (values(1,1) 
union all 
select level+1, mysum+mysum
from mytable where level<5)
select level, mysum
from mytable

 * db2+ibm_db://hloeser:***@localhost:50000/SAMPLE
Done.


level,mysum
1,1
2,2
3,4
4,8
5,16


## Tabellenabhängigkeit
* Ermittlung der Reihenfolge, in welcher Tabellen einer Datenbank zu importieren sind
* Reduziert Probleme mit Querverweisen / Referenzen
* Danke an Fiducia und GAD IT AG

In [37]:
%%sql 
WITH temp1 (tabname,reftable) AS
       (
         SELECT distinct rtrim(t.tabschema)||'.'||t.tabname,
                NULLIF(rtrim(r.reftabschema)||'.'||r.reftabname,
                rtrim(t.tabschema)||'.'||t.tabname) AS reftable
         FROM syscat.tables AS t
              LEFT OUTER JOIN syscat.references AS r
              ON t.tabschema = r.tabschema AND t.tabname = r.tabname
         WHERE t.tabschema NOT LIKE  'SYS%' AND type = 'T'),
     temp2 (table, reftable,lvl) AS
       (
         SELECT tabname,reftable,1
         FROM temp1
         WHERE reftable IS null
               AND tabname NOT IN (select tabname from temp1
                                   where reftable is not null)
         UNION ALL
         SELECT t.tabname,t.reftable,z.lvl+1
         FROM temp1 AS t, temp2 AS z
         WHERE t.reftable = z.table)
SELECT table,MAX(lvl) AS mlevel
FROM temp2
GROUP BY table
ORDER BY mlevel,table

 * db2+ibm_db://hloeser:***@localhost:50000/SAMPLE
Done.


TABLE,mlevel
HLOESER.ACT,1
HLOESER.CATALOG,1
HLOESER.CL_SCHED,1
HLOESER.CUSTOMER,1
HLOESER.EMPMDC,1
HLOESER.INVENTORY,1
HLOESER.IN_TRAY,1
HLOESER.ORG,1
HLOESER.PRODUCT,1
HLOESER.PRODUCTSUPPLIER,1


## Lösung eines Sudoku

In [9]:
%%sql sudoku << 
WITH 
  INPUT(sud) AS (
    VALUES('53..7....6..195....98....6.8...6...34..8.3..17...2...6.6....28....419..5....8..79')
  ),
  DIGITS(z, lp) AS (
    VALUES('1', 1)
    UNION ALL SELECT
    CAST(lp+1 AS varchar(200)), lp+1 FROM DIGITS WHERE lp<9
  ),
  X(s, ind) AS (
    SELECT sud, instr(sud, '.') FROM INPUT
    UNION ALL
    SELECT
      substr(s, 1, ind-1) || z || substr(s, ind+1),
      instr( substr(s, 1, ind-1) || z || substr(s, ind+1), '.' )
    FROM X, DIGITS AS z
    WHERE ind>0
      AND NOT EXISTS (
            SELECT 1
            FROM DIGITS AS lp
            WHERE z.z = substr(s, ((ind-1)/9)*9 + lp, 1)
                  OR z.z = substr(s, ((ind-1)%9) + (lp-1)*9 + 1, 1)
                  OR z.z = substr(s, (((ind-1)/3) % 3) * 3
                        + ((ind-1)/27) * 27 + lp
                        + ((lp-1) / 3) * 6, 1)
         )
  )
SELECT s as solution FROM X WHERE ind=0;


 * db2+ibm_db://hloeser:***@localhost:50000/SAMPLE
Done.
Returning data to local variable sudoku


In [58]:
print "Kleiner Test"
s2=sudoku.dict()['solution']
for y in range(0,9):
    if y in (0,3,6):
        print '+---+---+---+'
    line=''
    for z in range(0,3):
        line=line+'|'
        mypos=y*9+z*3
        line=line+(s2[0][mypos:3+mypos])
    print line+'|'
print '+---+---+---+'
    

Kleiner Test
+---+---+---+
|534|678|912|
|672|195|348|
|198|342|567|
+---+---+---+
|859|761|423|
|426|853|791|
|713|924|856|
+---+---+---+
|961|537|284|
|287|419|635|
|345|286|179|
+---+---+---+


## CTE mit XML-Daten
* Achtung: es ist sehr einfach, große Daten zu erzeugen

In [10]:
%%sql 
with mytable(level,mydoc) as
(values(1,xmlelement(name "bar", 'Hello world')) 
union all 
select level+1, xmlelement(name "foo", mydoc, mydoc)
from mytable where level<5)
select level, mydoc
from mytable
where level=5

 * db2+ibm_db://hloeser:***@localhost:50000/SAMPLE
Done.


level,mydoc
5,"﻿<?xml version=""1.0"" encoding=""UTF-16"" ?><foo><foo><foo><foo><bar>Hello world</bar><bar>Hello world</bar></foo><foo><bar>Hello world</bar><bar>Hello world</bar></foo></foo><foo><foo><bar>Hello world</bar><bar>Hello world</bar></foo><foo><bar>Hello world</bar><bar>Hello world</bar></foo></foo></foo><foo><foo><foo><bar>Hello world</bar><bar>Hello world</bar></foo><foo><bar>Hello world</bar><bar>Hello world</bar></foo></foo><foo><foo><bar>Hello world</bar><bar>Hello world</bar></foo><foo><bar>Hello world</bar><bar>Hello world</bar></foo></foo></foo></foo>"


In [15]:
%%sql 
with mytable(level,mydoc) as
(values(1,xmlelement(name "first", 'deep down - hello')) 
union all 
select level+1, 
 case when level <4 then 
 xmlelement(name "later", mydoc, mydoc)
else xmlelement(name "last", mydoc, 'the end is near...?!')
 end
from mytable where level<18)
select level, xmlelement(name "root",mydoc)
from mytable
where level=18

 * db2+ibm_db://hloeser:***@localhost:50000/SAMPLE
Done.


level,2
18,"﻿<?xml version=""1.0"" encoding=""UTF-16"" ?><root><last><last><last><last><last><last><last><last><last><last><last><last><last><last><later><later><later><first>deep down - hello</first><first>deep down - hello</first></later><later><first>deep down - hello</first><first>deep down - hello</first></later></later><later><later><first>deep down - hello</first><first>deep down - hello</first></later><later><first>deep down - hello</first><first>deep down - hello</first></later></later></later>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last>the end is near...?!</last></root>"


In [12]:
%%sql 
with mytable(level,mystmt)
as (values(1,cast 
('insert into foobar values (1, 
 ''Henrik was here'')' as varchar(500)) ) 
union all 
select level+1,
        mystmt || ',(' || level || ', ''Hello'')' 
from mytable where level<4)
select level, mystmt
from mytable

 * db2+ibm_db://hloeser:***@localhost:50000/SAMPLE
Done.


level,mystmt
1,"insert into foobar values (1, 'Henrik was here')"
2,"insert into foobar values (1, 'Henrik was here'),(1, 'Hello')"
3,"insert into foobar values (1, 'Henrik was here'),(1, 'Hello'),(2, 'Hello')"
4,"insert into foobar values (1, 'Henrik was here'),(1, 'Hello'),(2, 'Hello'),(3, 'Hello')"


Beispiel 7 aus der [Db2-Dokumentation mit SQL-Beispielen](
https://www.ibm.com/support/knowledgecenter/SSEPGG_11.1.0/com.ibm.db2.luw.sql.ref.doc/doc/r0059224.html):

`WITH
    NEWEMP AS (SELECT EMPNO FROM NEW TABLE
                (INSERT INTO EMPLOYEE(EMPNO, FIRSTNME)
                    VALUES(NEXT VALUE FOR EMPNO_SEQ, 'GEORGE'))),
    OLDEMP AS (SELECT EMPNO FROM EMPLOYEE WHERE FIRSTNME = 'SALLY'),
    UPPROJ AS (SELECT PROJNAME FROM NEW TABLE
                (UPDATE PROJECT
                    SET RESPEMP = (SELECT EMPNO FROM NEWEMP)
                    WHERE RESPEMP = (SELECT EMPNO FROM OLDEMP))),
    DELEMP AS (SELECT EMPNO FROM OLD TABLE
                (DELETE FROM EMPLOYEE
                    WHERE EMPNO = (SELECT EMPNO FROM OLDEMP)))
  SELECT PROJNAME FROM UPPROJ;`

## CONNECT BY (Oracle-Syntax)
Organisation der Abteilungen (DEPARTMENT) in der SAMPLE-DB

In [13]:
%%sql  
    SELECT LEVEL, CAST(SPACE((LEVEL - 1) * 4) || '/' || DEPTNAME
       AS VARCHAR(40)) AS DEPTNAME
     FROM DEPARTMENT
     START WITH DEPTNO = 'A00'
     CONNECT BY NOCYCLE PRIOR DEPTNO = ADMRDEPT

 * db2+ibm_db://hloeser:***@localhost:50000/SAMPLE
Done.


level,deptname
1,/SPIFFY COMPUTER SERVICE DIV.
2,/PLANNING
2,/INFORMATION CENTER
2,/DEVELOPMENT CENTER
3,/MANUFACTURING SYSTEMS
3,/ADMINISTRATION SYSTEMS
2,/SUPPORT SERVICES
3,/OPERATIONS
3,/SOFTWARE SUPPORT
3,/BRANCH OFFICE F2


## CTE
Organisation der Abteilungen (DEPARTMENT) in der SAMPLE-DB

In [14]:
%%sql  
with tdep(level, deptname, deptno) as (
    select 1, CAST( DEPTNAME AS VARCHAR(40)) AS DEPTNAME, deptno
    from department 
    where DEPTNO = 'A00'
    UNION ALL
    SELECT t.LEVEL+1, CAST(SPACE(t.LEVEL  * 4) || '/' || d.DEPTNAME
       AS VARCHAR(40)) AS DEPTNAME, d.deptno
     FROM DEPARTMENT d, tdep t
     where d.admrdept=t.deptno and d.deptno<>'A00')
select level, deptname
from tdep
     

 * db2+ibm_db://hloeser:***@localhost:50000/SAMPLE
Done.


level,deptname
1,SPIFFY COMPUTER SERVICE DIV.
2,/PLANNING
2,/INFORMATION CENTER
2,/DEVELOPMENT CENTER
2,/SUPPORT SERVICES
3,/MANUFACTURING SYSTEMS
3,/ADMINISTRATION SYSTEMS
3,/OPERATIONS
3,/SOFTWARE SUPPORT
3,/BRANCH OFFICE F2


# Stuff I used
***
* Db2: Common Table Expression https://www.ibm.com/support/knowledgecenter/SSEPGG_11.1.0/com.ibm.db2.luw.sql.ref.doc/doc/r0059217.html
* CONNECT BY: https://www.ibm.com/support/knowledgecenter/SSEPGG_11.1.0/com.ibm.db2.luw.apdv.porting.doc/doc/r0052877.html
* Bill of Material: https://www.ibm.com/support/knowledgecenter/SSEPGG_11.1.0/com.ibm.db2.luw.sql.ref.doc/doc/r0059242.html
* Db2 Sample Database: https://www.ibm.com/support/knowledgecenter/SSEPGG_11.1.0/com.ibm.db2.luw.apdv.samptop.doc/doc/r0001094.html
* https://de.wikipedia.org/wiki/Rekursion
* https://de.wikipedia.org/wiki/Mise_en_abyme
* SQL Magic / ipython-sql: https://github.com/catherinedevlin/ipython-sql
* ...